In [1]:
# Cell 1: Complete Voice Scam Shield System
import numpy as np
import sounddevice as sd
import whisper
import threading
import queue
import time
from datetime import datetime
import json
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Audio
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

class VoiceScamShield:
    def __init__(self):
        # Initialize components
        self.whisper_model = whisper.load_model("base")
        self.scam_detector = ScamDetector()  # From previous notebook
        self.antispoofing = SimpleAntiSpoofing()  # From previous notebook
        
        # Audio settings
        self.sample_rate = 16000
        self.chunk_duration = 3
        self.chunk_samples = int(self.chunk_duration * self.sample_rate)
        
        # State
        self.recording = False
        self.audio_queue = queue.Queue()
        self.full_transcript = ""
        self.alerts = []
        
        # Create UI
        self.create_dashboard()
        
    def create_dashboard(self):
        """Create the main dashboard UI"""
        # Control buttons
        self.start_btn = widgets.Button(
            description="🛡️ Start Protection",
            button_style='success',
            style={'button_color': 'green'}
        )
        self.stop_btn = widgets.Button(
            description="⏹️ Stop Protection",
            button_style='danger',
            disabled=True
        )
        
        # Status indicators
        self.status_label = widgets.HTML("<h2>🛡️ Voice Scam Shield - Ready</h2>")
        self.risk_indicator = widgets.HTML(
            "<div style='padding:10px; background-color:#e8f5e8; border-radius:5px;'>"
            "<h3>🟢 Status: SAFE</h3></div>"
        )
        
        # Transcript display
        self.transcript_output = widgets.Output()
        self.alerts_output = widgets.Output()
        
        # Event handlers
        self.start_btn.on_click(self.start_protection)
        self.stop_btn.on_click(self.stop_protection)
        
        # Layout
        controls = HBox([self.start_btn, self.stop_btn])
        
        self.dashboard = VBox([
            self.status_label,
            controls,
            self.risk_indicator,
            widgets.HTML("<h3>📝 Live Transcript:</h3>"),
            self.transcript_output,
            widgets.HTML("<h3>🚨 Security Alerts:</h3>"),
            self.alerts_output
        ])
    
    def start_protection(self, btn):
        """Start the voice scam protection"""
        self.recording = True
        self.start_btn.disabled = True
        self.stop_btn.disabled = False
        
        # Update status
        self.status_label.value = "<h2>🛡️ Voice Scam Shield - ACTIVE</h2>"
        
        with self.transcript_output:
            clear_output()
            print("🎤 Protection started! Monitoring your calls...")
            
        with self.alerts_output:
            clear_output()
            print("👁️ Watching for scam patterns...")
        
        # Start audio stream
        self.stream = sd.InputStream(
            callback=self.audio_callback,
            channels=1,
            samplerate=self.sample_rate,
            blocksize=self.chunk_samples
        )
        self.stream.start()
        
        # Start processing thread
        self.processing_thread = threading.Thread(target=self.processing_loop)
        self.processing_thread.start()
    
    def stop_protection(self, btn):
        """Stop the protection"""
        self.recording = False
        self.start_btn.disabled = False
        self.stop_btn.disabled = True
        
        self.status_label.value = "<h2>🛡️ Voice Scam Shield - Stopped</h2>"
        
        if hasattr(self, 'stream'):
            self.stream.stop()
            self.stream.close()
        
        self.update_risk_indicator("SAFE", 0.0)
    
    def audio_callback(self, indata, frames, time, status):
        """Audio stream callback"""
        if self.recording and not self.audio_queue.full():
            audio_chunk = indata[:, 0].copy()
            self.audio_queue.put(audio_chunk)
    
    def processing_loop(self):
        """Main processing loop"""
        while self.recording:
            try:
                if not self.audio_queue.empty():
                    audio_chunk = self.audio_queue.get(timeout=1)
                    
                    if len(audio_chunk) >= self.chunk_samples:
                        # Process the audio chunk
                        self.process_audio_chunk(audio_chunk)
                else:
                    time.sleep(0.1)
                    
            except queue.Empty:
                continue
            except Exception as e:
                with self.alerts_output:
                    print(f"❌ Processing error: {e}")
    
    def process_audio_chunk(self, audio_chunk):
        """Process a single audio chunk through the complete pipeline"""
        try:
            # Step 1: Transcription
            transcription_result = self.whisper_model.transcribe(
                audio_chunk,
                fp16=False,
                language=None
            )
            
            text = transcription_result["text"].strip()
            language = transcription_result["language"]
            
            if not text:
                return
            
            # Update transcript
            self.full_transcript += f"{text} "
            
            # Step 2: Anti-spoofing detection
            antispoofing_result = self.antispoofing.detect_synthetic_voice(audio_chunk)
            
            # Step 3: Scam detection
            scam_result = self.scam_detector.analyze_transcript(text, language)
            
            # Step 4: Risk assessment
            final_risk, combined_confidence = self.assess_combined_risk(
                scam_result, antispoofing_result
            )
            
            # Update UI
            self.update_transcript_display(text, language)
            self.update_risk_indicator(final_risk, combined_confidence)
            
            # Generate alerts if needed
            if final_risk in ["SUSPICIOUS", "SCAM"]:
                self.generate_alert(final_risk, scam_result, antispoofing_result)
                
        except Exception as e:
            with self.alerts_output:
                print(f"❌ Pipeline error: {e}")
    
    def assess_combined_risk(self, scam_result, antispoofing_result):
        """Combine scam detection and anti-spoofing results"""
        # Prioritize synthetic voice detection
        if antispoofing_result["is_synthetic"]:
            return "SCAM", max(0.8, antispoofing_result["confidence"])
        
        # Use scam detection result
        scam_risk = scam_result.get("risk_level", "SAFE")
        scam_confidence = scam_result.get("confidence", 0.0)
        
        return scam_risk, scam_confidence
    
    def update_transcript_display(self, new_text, language):
        """Update the transcript display"""
        with self.transcript_output:
            clear_output(wait=True)
            print(f"🌍 Language: {language.upper()}")
            print("-" * 50)
            print(self.full_transcript[-500:])  # Show last 500 chars
            print("-" * 50)
            print(f"Latest: {new_text}")
    
    def update_risk_indicator(self, risk_level, confidence):
        """Update the risk indicator"""
        colors = {
            "SAFE": ("#e8f5e8", "🟢", "green"),
            "SUSPICIOUS": ("#fff3cd", "🟡", "orange"),
            "SCAM": ("#f8d7da", "🔴", "red"),
            "UNKNOWN": ("#e2e3e5", "⚪", "gray")
        }
        
        bg_color, emoji, text_color = colors.get(risk_level, colors["UNKNOWN"])
        
        self.risk_indicator.value = f"""
        <div style='padding:15px; background-color:{bg_color}; border-radius:8px; border: 2px solid {text_color};'>
            <h3>{emoji} Risk Level: {risk_level}</h3>
            <p><strong>Confidence:</strong> {confidence:.1%}</p>
        </div>
        """
    
    def generate_alert(self, risk_level, scam_result, antispoofing_result):
        """Generate and display security alerts"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        
        alert = {
            "time": timestamp,
            "risk_level": risk_level,
            "scam_result": scam_result,
            "antispoofing_result": antispoofing_result
        }
        
        self.alerts.append(alert)
        
        with self.alerts_output:
            print(f"\n🚨 ALERT at {timestamp}")
            print(f"📊 Risk Level: {risk_level}")
            
            if antispoofing_result["is_synthetic"]:
                print("🤖 SYNTHETIC VOICE DETECTED!")
                
            if scam_result.get("detected_patterns"):
                print(f"⚠️ Scam Patterns: {', '.join(scam_result['detected_patterns'])}")
                
            print(f"💭 Analysis: {scam_result.get('reasoning', 'Unknown')}")
            print("-" * 30)

# Create and display the Voice Scam Shield
print("🛡️ Initializing Voice Scam Shield...")
shield = VoiceScamShield()
print("✅ Voice Scam Shield Ready!")

display(shield.dashboard)


🛡️ Initializing Voice Scam Shield...


NameError: name 'ScamDetector' is not defined

In [2]:
# Cell 1: Imports and Dependencies
import os
import json
import threading
import queue
import time
from datetime import datetime

import numpy as np
import sounddevice as sd
import whisper
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Audio
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Ensure ScamDetector and SimpleAntiSpoofing are defined or imported
# If they are defined in another notebook/module, import them:
# from scam_detector_module import ScamDetector
# from antispoofing_module import SimpleAntiSpoofing

# Otherwise, include their definitions here before using:
class ScamDetector:
    def __init__(self, api_key: str = None):
        import openai
        # Set up API key from env or argument
        if api_key:
            openai.api_key = api_key
            os.environ["OPENAI_API_KEY"] = api_key
        self.client = openai.Client(api_key=openai.api_key)
    def get_scam_prompt(self, language="english"):
        # (prompt implementation as before)
        return "<YOUR_PROMPT>"
    def analyze_transcript(self, transcript, language="english"):
        # (analysis implementation as before)
        return {"risk_level":"SAFE","confidence":1.0,"detected_patterns":[],"scam_type":"unknown","reasoning":""}

class SimpleAntiSpoofing:
    def __init__(self):
        pass
    def detect_synthetic_voice(self, audio_chunk):
        # Dummy implementation; replace with real logic
        return {"is_synthetic": False, "confidence": 0.0}


# Cell 2: VoiceScamShield Definition
class VoiceScamShield:
    def __init__(self):
        # Initialize components
        self.whisper_model = whisper.load_model("base")
        self.scam_detector = ScamDetector(api_key="sk-…")  
        self.antispoofing = SimpleAntiSpoofing()
        
        # Audio settings
        self.sample_rate = 16000
        self.chunk_duration = 3
        self.chunk_samples = int(self.chunk_duration * self.sample_rate)
        
        # State
        self.recording = False
        self.audio_queue = queue.Queue()
        self.full_transcript = ""
        self.alerts = []
        
        # Create UI
        self.create_dashboard()
        
    def create_dashboard(self):
        """Create the main dashboard UI"""
        self.start_btn = widgets.Button(description="🛡️ Start Protection", button_style='success')
        self.stop_btn = widgets.Button(description="⏹️ Stop Protection", button_style='danger', disabled=True)
        self.status_label = widgets.HTML("<h2>🛡️ Voice Scam Shield - Ready</h2>")
        self.risk_indicator = widgets.HTML(
            "<div style='padding:10px; background-color:#e8f5e8; border-radius:5px;'>"
            "<h3>🟢 Status: SAFE</h3></div>"
        )
        self.transcript_output = widgets.Output()
        self.alerts_output = widgets.Output()
        self.start_btn.on_click(self.start_protection)
        self.stop_btn.on_click(self.stop_protection)
        controls = HBox([self.start_btn, self.stop_btn])
        self.dashboard = VBox([
            self.status_label,
            controls,
            self.risk_indicator,
            widgets.HTML("<h3>📝 Live Transcript:</h3>"),
            self.transcript_output,
            widgets.HTML("<h3>🚨 Security Alerts:</h3>"),
            self.alerts_output
        ])
    
    def start_protection(self, btn):
        self.recording = True
        self.start_btn.disabled = True
        self.stop_btn.disabled = False
        self.status_label.value = "<h2>🛡️ Voice Scam Shield - ACTIVE</h2>"
        with self.transcript_output:
            clear_output(); print("🎤 Protection started! Monitoring your calls...")
        with self.alerts_output:
            clear_output(); print("👁️ Watching for scam patterns...")
        self.stream = sd.InputStream(callback=self.audio_callback,
                                     channels=1,
                                     samplerate=self.sample_rate,
                                     blocksize=self.chunk_samples)
        self.stream.start()
        self.processing_thread = threading.Thread(target=self.processing_loop, daemon=True)
        self.processing_thread.start()
    
    def stop_protection(self, btn):
        self.recording = False
        self.start_btn.disabled = False
        self.stop_btn.disabled = True
        self.status_label.value = "<h2>🛡️ Voice Scam Shield - Stopped</h2>"
        if hasattr(self, 'stream'):
            self.stream.stop(); self.stream.close()
        self.update_risk_indicator("SAFE", 0.0)
    
    def audio_callback(self, indata, frames, time, status):
        if self.recording and not self.audio_queue.full():
            self.audio_queue.put(indata[:, 0].copy())
    
    def processing_loop(self):
        while self.recording:
            try:
                audio_chunk = self.audio_queue.get(timeout=1)
                if len(audio_chunk) >= self.chunk_samples:
                    self.process_audio_chunk(audio_chunk)
            except queue.Empty:
                continue
    
    def process_audio_chunk(self, audio_chunk):
        try:
            # Transcription
            result = self.whisper_model.transcribe(audio_chunk, fp16=False)
            text = result["text"].strip()
            language = result.get("language", "unknown")
            if not text:
                return
            self.full_transcript += text + " "
            # Anti‐spoofing
            spoof = self.antispoofing.detect_synthetic_voice(audio_chunk)
            # Scam detection
            scam = self.scam_detector.analyze_transcript(text, language)
            # Risk assessment
            risk, conf = self.assess_combined_risk(scam, spoof)
            # Update UI
            self.update_transcript_display(text, language)
            self.update_risk_indicator(risk, conf)
            if risk in ["SUSPICIOUS", "SCAM"]:
                self.generate_alert(risk, scam, spoof)
        except Exception as e:
            with self.alerts_output:
                print(f"❌ Pipeline error: {e}")
    
    def assess_combined_risk(self, scam_result, antispoofing_result):
        if antispoofing_result["is_synthetic"]:
            return "SCAM", max(0.8, antispoofing_result["confidence"])
        return scam_result.get("risk_level", "SAFE"), scam_result.get("confidence", 0.0)
    
    def update_transcript_display(self, new_text, language):
        with self.transcript_output:
            clear_output(wait=True)
            print(f"🌍 Language: {language.upper()}")
            print("-" * 50)
            print(self.full_transcript[-500:])
            print("-" * 50)
            print(f"Latest: {new_text}")
    
    def update_risk_indicator(self, risk_level, confidence):
        colors = {
            "SAFE": ("#e8f5e8", "🟢", "green"),
            "SUSPICIOUS": ("#fff3cd", "🟡", "orange"),
            "SCAM": ("#f8d7da", "🔴", "red"),
            "UNKNOWN": ("#e2e3e5", "⚪", "gray")
        }
        bg, emoji, text_color = colors.get(risk_level, colors["UNKNOWN"])
        self.risk_indicator.value = f"""
        <div style='padding:15px; background-color:{bg}; border-radius:8px; border:2px solid {text_color};'>
            <h3>{emoji} Risk Level: {risk_level}</h3>
            <p><strong>Confidence:</strong> {confidence:.1%}</p>
        </div>
        """
    
    def generate_alert(self, risk_level, scam_result, antispoofing_result):
        ts = datetime.now().strftime("%H:%M:%S")
        with self.alerts_output:
            print(f"\n🚨 ALERT at {ts}")
            if antispoofing_result["is_synthetic"]:
                print("🤖 SYNTHETIC VOICE DETECTED!")
            if scam_result.get("detected_patterns"):
                print(f"⚠️ Scam Patterns: {', '.join(scam_result['detected_patterns'])}")
            print(f"💭 Analysis: {scam_result.get('reasoning','Unknown')}")
            print("-" * 30)


# Cell 3: Initialize and Display
print("🛡️ Initializing Voice Scam Shield...")
shield = VoiceScamShield()
print("✅ Voice Scam Shield Ready!")
display(shield.dashboard)


🛡️ Initializing Voice Scam Shield...
✅ Voice Scam Shield Ready!
